In [ ]:
import pyxdf
import matplotlib.pyplot as plt
import numpy as np
import itertools
import os
import mne
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from xgboost import XGBClassifier
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential,Input,Model,model_from_json
from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import accuracy_score
from keras.layers import Conv1D, Conv2D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GRU, LSTM, RNN, Dense, Dropout, Flatten, Activation, Add, ZeroPadding2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras import regularizers as reg
from keras.utils.vis_utils import model_to_dot, plot_model
from IPython.display import SVG
from keras.initializers import glorot_uniform
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
def baseline(data, data_t, ref_ival):
    '''
    Usage:
        To baseline the raw data before epoching
        data = baseline(data, data_t, ref_ival)
    Parameters:
        data: a 2D array of data array (n_chan * n_dataPoint)
        ref_ival: a two element vector specifying the time interval for which the baseline is calculated [ms]
    '''
    idxref = (data_t[0] + ref_ival[0] <= data_t) & (data_t <= data_t[0] + (0.001 * ref_ival[1])) # convert ms to s -> 1ms = 0.001s
    dataref = np.mean(data[:, idxref], axis=1, keepdims=True)
    data = data - dataref
    return data

In [ ]:
def prepare_train_and_test(data_with_label, train_ratio, test_ratio): 
    '''
    Usage:
        To shuffle and slice data points with cooresponding labels into training and testing set 
        training_data, training_label, testing_data, testing_label = prepare_train_and_test(data, label, train_ratio, test_ratio)
    Parameters:
        data_with_label: a 2D array of data array (n_dataPoint * (label + n_dementions)) with label at row 0
        train_ratio, test_ratio: training and testing ratios 
    '''
    # shuffling data with labels
    np.random.shuffle(data_with_label)
    shuffle_data = data_with_label[:, 1:]
    shuffle_label = data_with_label[:, 0]
    num_dataPoint = data_with_label.shape[0]
    
    # slicing
    training_data = shuffle_data[0: int(num_dataPoint * train_ratio)]
    training_label = shuffle_label[0: int(num_dataPoint * train_ratio)]
    testing_data = shuffle_data[int(num_dataPoint * test_ratio)-1: -1]
    testing_label = shuffle_label[int(num_dataPoint * test_ratio)-1: -1]
    
    return training_data, training_label, testing_data, testing_label

In [ ]:
def convert_to_raw(data, chan_name, sfreq, device):
    # load electrode locations
    enobio_chan_name = []
    if device == 'enobio' or device == 'Enobio':
        layout = pd.read_csv(r'C:\Users\m46ht\Desktop\EEGlass\Enobio_layout.txt', sep = '\t')
    elif device == 'EEGlasses' or device == 'obci':
        layout = pd.read_csv(r'C:\Users\m46ht\Desktop\EEGlass\EEGlasses_layout.txt', sep = '\t')
    layout.columns = layout.columns.str.strip()
    layout["Name"] = layout["labels"].str.strip()
    layout = layout.set_index('Name')
    layout = layout.to_dict(orient = "index")
    for channel in layout.keys():
        yxz = np.array([-1*layout[channel]["Y"], layout[channel]["X"], layout[channel]["Z"]])
        layout[channel] = yxz
        enobio_chan_name.append(channel)

    if device == 'enobio' or device == 'Enobio':
        # remove empty channel EXT
        layout.popitem()
    montage = mne.channels.make_dig_montage(layout, coord_frame='head')
    
    # create general info
    info = mne.create_info(chan_name, sfreq, ch_types='misc', verbose=False)
    data = data / 1e6 # convert units from V to muV
    raw = mne.io.RawArray(data, info)

    if device == 'enobio' or device == 'Enobio':
        raw.set_channel_types({chan_name[i]: 'eeg' for i in range(0, len(chan_name)-1)})
        raw.set_montage(montage)
    elif device == 'EEGlasses' or device == 'obci':
        raw.set_channel_types({chan_name[i]: 'eeg' for i in range(0, len(chan_name))})
        raw.set_montage(montage, on_missing='ignore')
    raw.load_data()
    # mark reference electrode as bad
    #raw.info['bads'] = ['EXT']
    raw.pick_types(meg=False, eeg=True, ref_meg=False)

    return raw

In [ ]:
def ICA_filter(raw, freqs_sig = [1, 90], n_components=20):  
    raw.load_data()
    filt_raw = raw.copy().filter(l_freq=freqs_sig[0], h_freq=freqs_sig[1])
    ica = mne.preprocessing.ICA(n_components, max_iter='auto', random_state=97, method='fastica')
    ica.fit(inst=filt_raw)
    return ica

In [ ]:
def apply_ICA_filter(ica, raw):
    raw.load_data()
    filtered_raw = ica.apply(inst = raw, verbose = False)
    return filtered_raw.get_data()

In [ ]:
def compute_psd(signal, fmin = 1., fmax = 90.):
    tmin = 0.1
    tmax = 5.
    sfreq = 8000
    psd, freqs = mne.time_frequency.psd_array_welch(
        signal, sfreq=sfreq, 
        n_fft=int(sfreq * (tmax - tmin)), n_overlap=0,
        n_per_seg=int(sfreq * (tmax - tmin)),fmin=fmin,
        fmax=fmax,window='boxcar',verbose=False)
    return psd, freqs

In [ ]:
enobio_chan_name = []
layout = pd.read_csv(r'C:\Users\m46ht\Desktop\EEGlass\Enobio_layout.txt', sep = '\t')
layout.columns = layout.columns.str.strip()
layout["Name"] = layout["labels"].str.strip()
layout = layout.set_index('Name')
layout = layout.to_dict(orient = "index")
for channel in layout.keys():
    #if channel == 'EXT':
    #    continue
    yxz = np.array([-1*layout[channel]["Y"], layout[channel]["X"], layout[channel]["Z"]])
    layout[channel] = yxz
    enobio_chan_name.append(channel)

# remove empty channel EXT
layout.popitem()
montage = mne.channels.make_dig_montage(layout, coord_frame='head')
montage_plot = mne.viz.plot_montage(montage, kind = 'topomap', scale_factor = 0.05, sphere=(0, 0, 0, 125))

In [ ]:
EEGlasses_chan_name = []
layout = pd.read_csv(r'C:\Users\m46ht\Desktop\EEGlass\EEGlasses_layout.txt', sep = '\t')
layout.columns = layout.columns.str.strip()
layout["Name"] = layout["labels"].str.strip()
layout = layout.set_index('Name')
layout = layout.to_dict(orient = "index")
for channel in layout.keys():
    #if channel == 'EXT':
    #    continue
    yxz = np.array([-1*layout[channel]["Y"], layout[channel]["X"], layout[channel]["Z"]])
    layout[channel] = yxz
    EEGlasses_chan_name.append(channel)

# remove empty channel EXT
layout.popitem()
montage = mne.channels.make_dig_montage(layout, coord_frame='head')
montage_plot = mne.viz.plot_montage(montage, kind = 'topomap', scale_factor = 0.05, sphere=(0, 0, 0, 125))

In [ ]:
def makeepochs(X, fs, markers, timestamp):
    '''
    Usage:
        makeepochs(X, fs, markers, ival)
    Parameters:
        X: 2D array of multi-channel timeseries (channels x samples) 
        fs: sampling frequency [Hz]
        markers: marker positions [sa]
        timestamp: time points for ongoing data (in s)
    Returns:
        epo: a 3D array of segmented signals (samples x channels x epochs)
        epo_t: a 1D array of time points of epochs relative to marker (in s)
    '''
    event_time_index = []
    epo_t = []
    #epo = np.array([])
    epo = []
    # 0-scaled time points for ongoing data 
    all_time_index = np.array([range(0, int(np.ceil((timestamp[-1] - timestamp[0])*fs))+1)])
    T = all_time_index.shape[1]
    nEvents = len(markers)
    markers_diff = np.diff(markers).tolist()
    nChans = X.shape[0]
    
    # generate the time points that events happend
    for i in range(len(markers_diff)):
        event_time_index.append(int(sum(markers_diff[0:i]) * fs))
    event_time_index.append(all_time_index[0][-1] + 1)

    #epo = X[:,idx].T.reshape(T, nEvents, nChans)
    #epo = np.transpose(epo, (0,2,1))
    #epo_t = np.linspace(0, timestamp[-1] - timestamp[0], T)
    #return epo, epo_t

    # slicing event epoch depending on markers' indice
    for j in range(len(event_time_index)): 
        if j != len(event_time_index)-1:
            event = X[:, event_time_index[j]:event_time_index[j+1]]
            #epo = np.append(epo, event, axis=0)
            epo.append(event)
            event_time = timestamp[j]
            epo_t.append(event_time)

    return epo, epo_t

In [ ]:
def make_epochs(X, time_series, mrk_pos, ival):
    '''
    Usage:
        makeepochs(X, fs, mrk_pos, ival)
    Parameters:
        X: 2D array of multi-channel timeseries (channels x samples) 
        fs: sampling frequency [Hz]
        mrk_pos: marker positions [sa]
        ival: a two element vector giving the time interval relative to markers (in ms)
    Returns:
        epo: a 3D array of segmented signals (samples x channels x epochs)
        epo_t: a 1D array of time points of epochs relative to marker (in ms)
    '''
    time = np.array([range(np.int(np.floor(ival[0]*fs/1000)), 
                           np.int(np.ceil(ival[1]*fs/1000))+1)])
    T = time_series.shape[1]
    nEvents = len(mrk_pos)
    nChans = X.shape[0]
    idx = (time.T+np.array([mrk_pos])).reshape(1, T*nEvents)    
    epo = X[:,idx].T.reshape(T, nEvents, nChans)
    epo = np.transpose(epo, (0,2,1))
    epo_t = np.linspace(ival[0], ival[1], T)
    return epo, epo_t

In [ ]:
taskType_list = ['2-back', 'Backward1', 'Backward2', 'Stroop1', 'Stroop2', 'Stroop3', 'Stroop4']
label_encoder = LabelEncoder()
taskType_integer_encoded = label_encoder.fit_transform(taskType_list)
# convert to one_hot
taskType_one_hot_encoded = np.zeros((taskType_integer_encoded.size, taskType_integer_encoded.max()+1))
taskType_one_hot_encoded[np.arange(taskType_integer_encoded.size),taskType_integer_encoded] = 1 

In [ ]:
def load_participant_data(parti_ind = [1], one_hot_encoding = True, scaling = True):
    #taskType_list = ['2-back', 'Backward1', 'Backward2', 'RestingClosed', 'RestingOpen', 'Stroop1', 'Stroop2', 'Stroop3', 'Stroop4']
    taskType_list = ['2-back', 'Backward1', 'Backward2', 'Stroop1', 'Stroop2', 'Stroop3', 'Stroop4']
    #label_encoder = LabelEncoder()
    #taskType_integer_encoded = label_encoder.fit_transform(taskType_list)
    taskType_integer_encoded = np.array([0, 1, 1, 2, 2, 2, 2]) # simplified into 3 classes: 2back, backward and stroop
    # convert to one_hot
    taskType_one_hot_encoded = np.zeros((taskType_integer_encoded.size, taskType_integer_encoded.max()+1))
    taskType_one_hot_encoded[np.arange(taskType_integer_encoded.size),taskType_integer_encoded] = 1 
    exclude = []
    participant_data_enobio = pd.DataFrame()
    participant_label_enobio = []
    participant_data_eegGlasses = pd.DataFrame()
    participant_label_eegGlasses = []
    # scaling into unit variance for ML classifiers
    min_max_scaler = MinMaxScaler()
    for participantID in parti_ind:
        if participantID in exclude:
            print("Contains dead channels... Please re-enter...")
            return [], [], [], []
        else:
            if one_hot_encoding:
                for taskType, taskType_one_hot in zip(taskType_list, taskType_one_hot_encoded): 
                    # read csv data 
                    df_enobio = pd.read_csv(r"C:\Users\m46ht\Desktop\EEGlass\enobio_psd_results\parti"
                                            + str(participantID) + "\parti_" + str(participantID) 
                                            + "_" + taskType + "_psd.csv")
                    df_eegGlasses = pd.read_csv(r"C:\Users\m46ht\Desktop\EEGlass\eegGlasses_psd_results\parti" 
                                                + str(participantID) + "\parti_" + str(participantID) 
                                                + "_" + taskType + "_psd.csv")
                    # drop the last row -- we have 19 active channels in each task 
                    df_enobio.drop(df_enobio.tail(1).index, inplace=True)
                    participant_data_enobio = participant_data_enobio.append(df_enobio)
                    participant_data_eegGlasses = participant_data_eegGlasses.append(df_eegGlasses)
                    for i in range(len(df_enobio)):
                        participant_label_enobio.append(taskType_one_hot)
                    for j in range(len(df_eegGlasses)):
                        participant_label_eegGlasses.append(taskType_one_hot)
            else: 
                for taskType, taskType_int in zip(taskType_list, taskType_integer_encoded): 
                    df_enobio = pd.read_csv(r"C:\Users\m46ht\Desktop\EEGlass\enobio_psd_results\parti" 
                                            + str(participantID) + "\parti_" + str(participantID) 
                                            + "_" + taskType + "_psd.csv")
                    df_eegGlasses = pd.read_csv(r"C:\Users\m46ht\Desktop\EEGlass\eegGlasses_psd_results\parti" 
                                                + str(participantID) + "\parti_" + str(participantID) 
                                                + "_" + taskType + "_psd.csv")
                    df_enobio.drop(df_enobio.tail(1).index, inplace=True)
                    participant_data_enobio = participant_data_enobio.append([df_enobio])
                    participant_data_eegGlasses = participant_data_eegGlasses.append([df_eegGlasses])
                    for i in range(len(df_enobio)):
                        participant_label_enobio.append([taskType_int])
                    for j in range(len(df_eegGlasses)):
                        participant_label_eegGlasses.append([taskType_int])

    participant_data_enobio = np.asarray(participant_data_enobio)
    participant_label_enobio = np.asarray(participant_label_enobio)
    participant_data_eegGlasses = np.asarray(participant_data_eegGlasses)
    participant_label_eegGlasses = np.asarray(participant_label_eegGlasses)
    
    if scaling: 
        participant_data_enobio = min_max_scaler.fit_transform(participant_data_enobio)
        participant_data_eegGlasses = min_max_scaler.fit_transform(participant_data_eegGlasses)
        return participant_data_enobio.tolist(), participant_label_enobio.tolist(), participant_data_eegGlasses.tolist(), participant_label_eegGlasses.tolist()
    else: 
        return participant_data_enobio.tolist(), participant_label_enobio.tolist(), participant_data_eegGlasses.tolist(), participant_label_eegGlasses.tolist()

In [ ]:
def plot_topomap(raw, l_freq, h_freq, plot_range, task):
    range_index = [int(raw.info['sfreq'] * (plot_range[0] - l_freq)), int(raw.info['sfreq'] * (plot_range[1] - l_freq))]
    data = np.mean(raw.get_data()[:, range_index[0] : range_index[1]], axis = 1)
    print("Topography for averaged value from " + str(plot_range) + " Hz")
    fig, ax = plt.subplots()
    mne.viz.plot_topomap(data, raw.info, outlines='head', extrapolate = 'head', sphere=(0, 0, 0, 125), axes=ax,
                     show=False)
    # add titles
    #ax.set_title(str(plot_range[0]) + "Hz to " + str(plot_range[1]) + "Hz -- " + str(task), fontweight='bold')
    return fig

In [ ]:
all_participants = np.arange(1, 35)
enobio_data = np.array([])
enbobio_label = []
fmin = 30.
fmax = 100.
ref_ival = [0, 500] # use 0 to 100ms as reference for baselining
for participantID in all_participants:
    for taskType, taskType_integer in zip(taskType_list, taskType_integer_encoded): 
        xdf_name = r"C:\Users\m46ht\Desktop\EEGlass\EEGlass-MasterDataset\EEGlass-MasterDataset\Participant-" + str(participantID) + "\EEG\\" + "" + taskType + "\\block_Default.xdf"
        stream, header = pyxdf.load_xdf(xdf_name)
        for item in stream:

            # collect data from enobio 20 channels 
            if item['info']['name'][0] == 'Enobio' and item['info']['type'][0] == 'EEG' and item['info']['channel_count'][0] == '20' and item["time_series"].T[0:3].shape[1] != 1:
                enobio_time = item["time_stamps"]                
                temp_enobio_data = item["time_series"].T
                #enobio_chan_type = [chan_type['type'] for chan_type in item['info']['desc'][0]['channel']]
                #enobio_chan_type = [chan_type[0] for chan_type in enobio_chan_type]
                # get sampling freq
                enobio_sfreq = float(item["info"]["nominal_srate"][0])

                # baselining
                temp_enobio_data = baseline(temp_enobio_data, enobio_time, ref_ival)
                temp_enobio_raw = convert_to_raw(temp_enobio_data, enobio_chan_name, enobio_sfreq, device = 'Enobio')
                enobio_psd = temp_enobio_raw.plot_psd(average = False, fmin = fmin, fmax = fmax)
                enobio_psd.savefig(r"C:\Users\m46ht\Desktop\EEGlass\enobio_psd_results\parti" + str(participantID) + "\parti_" + str(participantID) + "_" + taskType + "_psd.png", dpi = 150, bbox_inches='tight') 
                
                # remove refrence channel -- they are all 0s
                #removed_refchan_data = rereferenced_raw.get_data()[~np.all(rereferenced_raw.get_data() == 0, axis=1)]
                removed_refchan_data = temp_enobio_raw.get_data()[~np.all(temp_enobio_raw.get_data() == 0, axis=1)]
                
                # conpute psd and their freqs 
                temp_enobio_psd, temp_enobio_freqs = compute_psd(removed_refchan_data, fmin = fmin, fmax = fmax)
                
                # name of csv file 
                temp_filename = r"C:\Users\m46ht\Desktop\EEGlass\enobio_psd_results\parti" + str(participantID) + "\parti_" + str(participantID) + "_" + taskType + "_psd.csv"
                df_psd = pd.DataFrame(temp_enobio_psd)
                df_psd.to_csv(temp_filename, index=False)
                
                
            # collect data from open bci 3 channels 
            elif (item['info']['name'][0] == 'obci_eeg1' or item['info']['name'][0] == 'openbci_eeg') and item['info']['type'][0] == 'EEG' and item['info']['channel_count'][0] == '8' and item["time_series"].T[0:3].shape[1] != 1:
 
                temp_obci_data = item["time_series"].T[0:3]
                obci_time = item["time_stamps"]
                obci_chan_name = ['Nz', 'TP10', 'TP9']
                obci_sfreq = float(item["info"]["nominal_srate"][0])
 
                temp_obci_data = baseline(temp_obci_data, obci_time, ref_ival)
                # for visualization only
                temp_obci_data = temp_obci_data / 700
                temp_obci_raw = convert_to_raw(temp_obci_data, obci_chan_name, obci_sfreq, device = 'EEGlasses')

                # collect psd plots
                #obci_psd = temp_obci_raw.plot_psd(average = False, fmin = fmin, fmax = fmax, exclude = ['Nz']) 
                obci_psd = temp_obci_raw.plot_psd(average = False, fmin = fmin, fmax = fmax) 
                obci_psd.savefig(r"C:\Users\m46ht\Desktop\EEGlass\eegGlasses_psd_results\parti" + str(participantID) + "\parti_" + str(participantID) + "_" + taskType + "_psd.png", dpi = 150, bbox_inches='tight') 

                # remove refrence channel -- they are all 0s
                #removed_refchan_data = rereferenced_raw.get_data()[~np.all(rereferenced_raw.get_data() == 0, axis=1)]
                removed_refchan_data = temp_obci_raw.get_data()[~np.all(temp_obci_raw.get_data() == 0, axis=1)]
                # conpute psd and their freqs 
                temp_obci_psd, temp_obci_freqs = compute_psd(removed_refchan_data, fmin = fmin, fmax = fmax)
                temp_filename = r"C:\Users\m46ht\Desktop\EEGlass\eegGlasses_psd_results\parti" + str(participantID) + "\parti_" + str(participantID) + "_" + taskType + "_psd.csv"
                df_psd = pd.DataFrame(temp_obci_psd)
                df_psd.to_csv(temp_filename, index=False)

In [ ]:
def cnn(train_X, train_label, conv_layers=1, conv_size=344, filter_size=1, fc_layers=2, fc_sizes=(4096,2048),
        dropout=0.5, pool_size=2, init='he_uniform', act='relu', optim='adam', pool=True,
        reg = reg.l2(0.05)):
    classifier = tf.keras.models.Sequential()
    for i in range(conv_layers):
        classifier.add(Conv1D(conv_size, filter_size, activation='relu',input_shape=(1, conv_size), kernel_initializer=init, kernel_regularizer=reg))
        classifier.add(BatchNormalization())
        if pool:
            classifier.add(MaxPooling1D(pool_size = 1))
    #classifier.add(Flatten())
    for j in range(fc_layers):
        classifier.add(Dense(fc_sizes, activation = act,kernel_initializer=init,kernel_regularizer=reg))
    #    classifier.add(Dropout(dropout))
    classifier.add(Dense(3, activation = 'softmax',kernel_initializer=init))
    classifier.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    classifier.fit(train_X, train_label, batch_size= 256, epochs= 15, verbose = True) 
    #classifier.summary()
    return classifier

In [ ]:
def concatenate(X, label, X_cat, label_cat):
    if len(X_cat) == 0:
        X_cat = X
        label_cat = label
    else:
        X_cat = np.concatenate((X_cat, X), axis = 0)
        label_cat = np.concatenate((label_cat, label), axis = 0)
    return X_cat, label_cat

In [ ]:
def get_test_result(participants, excluded_participants, individual_train = True, test_item = 'Enobio', test_model = 'svm', scaling = False):
    acc_val_list = []
    # create a list of random seeds for getting reasonable acc vals 
    # -- (K-fold val like but due to short in data, it is hard to implement such)
    rand_seed_list = [1]
    
    if test_model == 'svm':
        svm_classifier = svm.SVC(kernel='poly')
        # train using data from indivudual seperately
        if individual_train:
            temp_acc_list = []
            for rand_seed in rand_seed_list:
                # initiation for combined training and testing 
                train_X_cat = np.array(())
                train_label_cat = np.array(())
                valid_X_cat = np.array(())
                valid_label_cat = np.array(())
                # train part
                for person in participants:
                    participant_data_enobio, participant_label_enobio, participant_data_eegGlasses, participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        train_X, train_label = participant_data_enobio, participant_label_enobio
                    elif test_item == 'EEGlasses':
                        train_X, train_label = participant_data_eegGlasses, participant_label_eegGlasses
                    train_X_cat, train_label_cat = concatenate(train_X, train_label, train_X_cat, train_label_cat)

                # valid/test part
                for person in excluded_participants:
                    excluded_participant_data_enobio, excluded_participant_label_enobio, excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        valid_X, valid_label = excluded_participant_data_enobio, excluded_participant_label_enobio
                    elif test_item == 'EEGlasses':
                        valid_X, valid_label = excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses
                    valid_X_cat, valid_label_cat = concatenate(valid_X, valid_label, valid_X_cat, valid_label_cat)
                
                # use concatenated data to fit classifier 
                svm_classifier.fit(train_X_cat,train_label_cat)

                yhat = svm_classifier.predict(valid_X_cat)
                acc_val = accuracy_score(valid_label_cat,yhat)
            acc_val_list = acc_val
    
    elif test_model == 'knn':
        knn_classifier = KNeighborsClassifier(n_neighbors = 1, weights = 'distance')
        # train using data from indivudual seperately
        if individual_train:
            temp_acc_list = []
            for rand_seed in rand_seed_list:
                # initiation for combined training and testing 
                train_X_cat = np.array(())
                train_label_cat = np.array(())
                valid_X_cat = np.array(())
                valid_label_cat = np.array(())
                # train part
                for person in participants:
                    participant_data_enobio, participant_label_enobio, participant_data_eegGlasses, participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        train_X, train_label = participant_data_enobio, participant_label_enobio
                    elif test_item == 'EEGlasses':
                        train_X, train_label = participant_data_eegGlasses, participant_label_eegGlasses
                    train_X_cat, train_label_cat = concatenate(train_X, train_label, train_X_cat, train_label_cat)
                
                # valid\test part
                for person in excluded_participants:
                    excluded_participant_data_enobio, excluded_participant_label_enobio, excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        valid_X, valid_label = excluded_participant_data_enobio, excluded_participant_label_enobio
                    elif test_item == 'EEGlasses':
                        valid_X, valid_label = excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses
                    valid_X_cat, valid_label_cat = concatenate(valid_X, valid_label, valid_X_cat, valid_label_cat)
                     
                # use concatenated data to fit classifier 
                knn_classifier.fit(train_X_cat, train_label_cat)
                num_ValdataPoint = len(valid_X_cat[0])
                for i in range(1, len(excluded_participants)+1):
                    #valid_X = valid_X_cat[int(num_ValdataPoint / len(excluded_participants) * (i-1)) : int(num_ValdataPoint / len(participants) * i)]
                    #valid_label = valid_label_cat[int(num_ValdataPoint / len(excluded_participants) * (i-1)) : int(num_ValdataPoint / len(excluded_participants) * i)]
                    yhat = knn_classifier.predict(valid_X_cat)
                    acc_val = accuracy_score(valid_label_cat, yhat)
                    temp_acc_list.append(acc_val)
                acc_val_list = temp_acc_list
                
                
    elif test_model == 'XGBoost':
        XGB_model = XGBClassifier()
        # train using data from indivudual seperately
        if individual_train:
            temp_acc_list = []
            for rand_seed in rand_seed_list:
                # initiation for combined training and testing 
                train_X_cat = np.array(())
                train_label_cat = np.array(())
                valid_X_cat = np.array(())
                valid_label_cat = np.array(())
                # train part
                for person in participants:
                    participant_data_enobio, participant_label_enobio, participant_data_eegGlasses, participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        train_X, train_label = participant_data_enobio, participant_label_enobio
                    elif test_item == 'EEGlasses':
                        train_X, train_label = participant_data_eegGlasses, participant_label_eegGlasses
                    train_X_cat, train_label_cat = concatenate(train_X, train_label, train_X_cat, train_label_cat)
                
                # valid\test part
                for person in excluded_participants:
                    excluded_participant_data_enobio, excluded_participant_label_enobio, excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = False, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        valid_X, valid_label = excluded_participant_data_enobio, excluded_participant_label_enobio
                    elif test_item == 'EEGlasses':
                        valid_X, valid_label = excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses
                    valid_X_cat, valid_label_cat = concatenate(valid_X, valid_label, valid_X_cat, valid_label_cat)
                    
                    
                # use concatenated data to fit classifier 
                XGB_model.fit(train_X_cat, train_label_cat)
                num_ValdataPoint = len(valid_X_cat[0])
                for i in range(1, len(excluded_participants)+1):
                    #valid_X = valid_X_cat[int(num_ValdataPoint / len(participants) * (i-1)) : int(num_ValdataPoint / len(participants) * i)]
                    #valid_label = valid_label_cat[int(num_ValdataPoint / len(participants) * (i-1)) : int(num_ValdataPoint / len(participants) * i)]
                    yhat = XGB_model.predict(valid_X_cat)
                    acc_val = accuracy_score(valid_label_cat, yhat)
                    temp_acc_list.append(acc_val)
                acc_val_list = temp_acc_list

    elif test_model == 'cnn':
        # train using data from indivudual seperately
        if individual_train:
            temp_acc_list = []
            for rand_seed in rand_seed_list:
                # initiation for combined training and testing 
                train_X_cat = np.array(())
                train_label_cat = np.array(())
                valid_X_cat = np.array(())
                valid_label_cat = np.array(())
                # train part
                for person in participants:
                    participant_data_enobio, participant_label_enobio, participant_data_eegGlasses, participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = True, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        train_X, train_label = participant_data_enobio, participant_label_enobio
                    elif test_item == 'EEGlasses':
                        train_X, train_label = participant_data_eegGlasses, participant_label_eegGlasses
                    train_X_cat, train_label_cat = concatenate(train_X, train_label, train_X_cat, train_label_cat)
                    
                for person in excluded_participants:
                    excluded_participant_data_enobio, excluded_participant_label_enobio, excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses = load_participant_data(parti_ind = [person], one_hot_encoding = True, scaling = scaling)
                    if test_item == 'Enobio' or test_item == 'enobio':
                        valid_X, valid_label = excluded_participant_data_enobio, excluded_participant_label_enobio
                    elif test_item == 'EEGlasses':
                        valid_X, valid_label = excluded_participant_data_eegGlasses, excluded_participant_label_eegGlasses
                    valid_X_cat, valid_label_cat = concatenate(valid_X, valid_label, valid_X_cat, valid_label_cat)
                    
                # use concatenated data to fit classifier 
                num_ValdataPoint = len(valid_X_cat[0])
                train_X_cat = np.expand_dims(train_X_cat, axis = 1)
                train_label_cat = np.expand_dims(train_label_cat, axis = 1)
                classifier = cnn(train_X_cat, train_label_cat, conv_layers=3, fc_layers=2, conv_size=344, filter_size=1, fc_sizes=344)
                for i in range(1, len(excluded_participants)+1):
                    
                    valid_X = valid_X_cat[int(num_ValdataPoint / len(excluded_participants) * (i-1)) : int(num_ValdataPoint / len(excluded_participants) * i)]
                    valid_label = valid_label_cat[int(num_ValdataPoint / len(excluded_participants) * (i-1)) : int(num_ValdataPoint / len(excluded_participants) * i)]
                    valid_X = np.expand_dims(valid_X, axis = 1)
                    valid_label = np.expand_dims(valid_label, axis = 1)
                    yhat = classifier.predict(valid_X)
                    valid_label = np.squeeze(valid_label, axis = 1)
                    yhat = np.squeeze(yhat, axis = 1)
                    yhat_clean = np.zeros(valid_label.shape)
                    yhat_clean[np.arange(yhat.shape[0]), np.argmax(yhat, axis=1)] = 1
                    acc_val = accuracy_score(np.argmax(valid_label, axis=1), np.argmax(yhat_clean, axis=1))
                    temp_acc_list.append(acc_val)                    

                    #acc_val = accuracy_score(valid_label, yhat)
                    #temp_acc_list.append(acc_val)
                acc_val_list = temp_acc_list
    return participants, acc_val_list